In [1]:
# Packages for data analysis
import pandas as pd
import numpy as np

In [2]:
pip install tweet-preprocessor

Note: you may need to restart the kernel to use updated packages.


In [2]:
df=pd.read_csv('data/100tweets.csv', sep=';')
print(df.shape)
df.head()

(100, 18)


,Column1,...1,Row.names,rowid,lat2,lon2,user_id,Column2,text2,retweeted,lang,id,created_at,noChar,iso_lang_1,prob_1,text1,classname
0,4,53090,93493,319774,49.302954,7.466072,434326976,NaN,"Not good, not bad...disappointing",False,en,"1,19E+18",29.10.2019 16:55,34,en,0.881157,"Not good, not bad...disappointing\r\n#MartinaP...",home
1,6,4340,117041,460051,49.641076,6.06006735,603021612,NaN,thats so poopoo,False,en,"3,30E+17",03.05.2013 16:03,15,en,0.862470,thats so poopoo,other
2,8,9104,145546,679738,49.611273,6.12770012,1298568337,NaN,"Monday, June , : Spaghetti Bolognese",False,it,"3,42E+17",03.06.2013 11:22,37,en,0.914869,"Monday, June 3, 2013: Spaghetti Bolognese http...",home
3,24,23743,25333,105500,49.658195,6.14052043,28345806,NaN,Where are the people who left on the boat?? Do...,False,en,"5,63E+17",05.02.2015 21:53,70,en,0.964202,Where are the people who left on the boat?? Do...,home
4,38,55020,97806,347149,63.833300,20.25,456045381,NaN,Christmas cosiness. Merry very. |,False,sv,"8,13E+17",24.12.2016 21:21,34,en,0.924872,Christmas cosiness. Merry very. | #christmasev...,home


In [4]:
import preprocessor as prep

In [5]:
## Set options for the tweet-preprocessor
prep.set_options(prep.OPT.URL, prep.OPT.RESERVED, prep.OPT.EMOJI, prep.OPT.SMILEY)

## Clean text and remove #,@ symbols
def clean_tweet(text):
    text = prep.clean(text)
    table = str.maketrans('','','#@')
    return text.translate(table)

In [6]:
df['text_processed'] = df['text1'].apply(lambda x: clean_tweet(x))

In [7]:
print(df.shape)
df.head()

(100, 19)


,Column1,...1,Row.names,rowid,lat2,lon2,user_id,Column2,text2,retweeted,lang,id,created_at,noChar,iso_lang_1,prob_1,text1,classname,text_processed
0,0,24121,26215,108414,51.329654,12.373450300000002,29430823,NaN,(Senior) Digital Product Designer in,False,ca,"1,18E+18",26.09.2019 12:55,37,en,0.828701,(Senior) Digital Product Designer in #Leipzig ...,work,(Senior) Digital Product Designer in Leipzig (...
1,1,5037,120680,485238,52.308321,4.76523399,632420479,NaN,Jugging down my nd Liter of coffee for today. ...,False,en,"3,27E+17",24.04.2013 14:25,57,en,0.965668,Jugging down my 2nd Liter of coffee for today....,transport,Jugging down my 2nd Liter of coffee for today....
2,2,12531,156862,755079,49.104809,6.196049299999999,2159817706,NaN,Happy Weekend ??,False,en,"9,54E+17",19.01.2018 10:50,17,en,0.818342,Happy Weekend ?? #Metz #Lorraine https://t.co/...,services,Happy Weekend ?? Metz Lorraine
3,3,44008,69312,235610,49.248500,-123.1088,215292728,NaN,"Good morning, Vancouver! :)",False,en,"1,18E+18",02.10.2019 14:13,28,en,0.740567,"Good morning, Vancouver! :) #Sunrise 07:13, no...",services,"Good morning, Vancouver! Sunrise 07:13, noon 1..."
4,4,53090,93493,319774,49.302954,7.466072,434326976,NaN,"Not good, not bad...disappointing",False,en,"1,19E+18",29.10.2019 16:55,34,en,0.881157,"Not good, not bad...disappointing\r\n#MartinaP...",home,"Not good, not bad...disappointing MartinaPadbe..."


In [3]:
# transforming column with tweets to the list
sentences = list(df['text1'].values) # text will be the input to the model

In [4]:
# Do not forget to use GPU in Colab
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [5]:
from transformers import pipeline

In [6]:
# Applying classifier to the list of tweets
activity_classifier = pipeline("zero-shot-classification", model = 'joeddav/xlm-roberta-large-xnli')
activity_result = activity_classifier(list(sentences), candidate_labels = [
'Sleep waking up', 'Baby infant family', 'Reading books', 'TV YouTube Netflix', 'Dish recipy kitchen',

'Campus school university education study','Work job','Training secondment traineeship',
'Office lunch colleagues department director','Business industry research conferences and forums', 

'Outdoor sports recreation','Nature landscape wildlife','Walk hiking','Weather season rain cold',

'Delays passenger arrival departure','Cycling bikes scooters','Airport waiting lounge','Car petrol traffic',

'Entertainment','Religion','Groceries and shopping','Healthcare and well-being',
'Fitness club', 'Services brand retail','Café restaurants','City sightseeing','Art shows performances',

'Politics and affairs','Personality and relationships','Beliefs wishes kindness'],)
activity_result[10]

Downloading:   0%|          | 0.00/734 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.09G [00:00<?, ?B/s]

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

{'sequence': 'It was a real inspirational work? https://t.co/dJyTH7EG2I',
 'labels': ['Work job',
  'Beliefs wishes kindness',
  'Art shows performances',
  'Delays passenger arrival departure',
  'Weather season rain cold',
  'Training secondment traineeship',
  'Personality and relationships',
  'Nature landscape wildlife',
  'Reading books',
  'Entertainment',
  'Sleep waking up',
  'Baby infant family',
  'Outdoor sports recreation',
  'Car petrol traffic',
  'City sightseeing',
  'Campus school university education study',
  'Business industry research conferences and forums',
  'Politics and affairs',
  'Walk hiking',
  'Dish recipy kitchen',
  'Religion',
  'Healthcare and well-being',
  'Cycling bikes scooters',
  'TV YouTube Netflix',
  'Services brand retail',
  'Airport waiting lounge',
  'Office lunch colleagues department director',
  'Groceries and shopping',
  'Café restaurants',
  'Fitness club'],
 'scores': [0.7197009921073914,
  0.026250706985592842,
  0.0166571047157

In [7]:
np.argmax(activity_result[0]['scores']) 

0

In [8]:
# Leaving only first prediction for validation
y_activity_pred = [result['labels'][np.argmax(result['scores'])] for result in activity_result]
y_activity_pred[10]

'Work job'

In [9]:
# Replacing predictions with general categories for validation
for i in range(len(y_activity_pred)):
  
    if y_activity_pred[i] == 'Sleep waking up':
        y_activity_pred[i] = 'home'    
    if y_activity_pred[i] == 'Baby infant family':
        y_activity_pred[i] = 'home' 
    if y_activity_pred[i] == 'Reading books':
        y_activity_pred[i] = 'home' 
    if y_activity_pred[i] == 'TV YouTube Netflix':
        y_activity_pred[i] = 'home' 
    if y_activity_pred[i] == 'Dish recipy kitchen':
        y_activity_pred[i] = 'home' 

    if y_activity_pred[i] == 'Campus school university education study':
        y_activity_pred[i] = 'work'
    if y_activity_pred[i] == 'Work job':
        y_activity_pred[i] = 'work'
    if y_activity_pred[i] == 'Training secondment traineeship':
        y_activity_pred[i] = 'work'
    if y_activity_pred[i] == 'Office lunch colleagues department director':
        y_activity_pred[i] = 'work'
    if y_activity_pred[i] == 'Business industry research conferences and forums':
        y_activity_pred[i] = 'work'

    if y_activity_pred[i] == 'Outdoor sports recreation':
        y_activity_pred[i] = 'outdoors'
    if y_activity_pred[i] == 'Nature landscape wildlife':
        y_activity_pred[i] = 'outdoors'
    if y_activity_pred[i] == 'Weather season rain cold':
        y_activity_pred[i] = 'outdoors'
    if y_activity_pred[i] == 'Walk hiking':
        y_activity_pred[i] = 'outdoors'

    if y_activity_pred[i] == 'Delays passenger arrival departure':
        y_activity_pred[i] = 'transport'
    if y_activity_pred[i] == 'Cycling bikes scooters':
        y_activity_pred[i] = 'transport'
    if y_activity_pred[i] == 'Airport waiting lounge':
        y_activity_pred[i] = 'transport'
    if y_activity_pred[i] == 'Car petrol traffic':
        y_activity_pred[i] = 'transport'
        
    if y_activity_pred[i] == 'Entertainment':
        y_activity_pred[i] = 'services'
    if y_activity_pred[i] == 'Religion':
        y_activity_pred[i] = 'services'
    if y_activity_pred[i] == 'Groceries and shopping':
        y_activity_pred[i] = 'services'
    if y_activity_pred[i] == 'Healthcare and well-being':
        y_activity_pred[i] = 'services'
    if y_activity_pred[i] == 'Fitness club':
        y_activity_pred[i] = 'services'
    if y_activity_pred[i] == 'Services brand retail':
        y_activity_pred[i] = 'services'
    if y_activity_pred[i] == 'Café restaurants':
        y_activity_pred[i] = 'services'
    if y_activity_pred[i] == 'City sightseeing':
        y_activity_pred[i] = 'services'
    if y_activity_pred[i] == 'Theatre shows performances':
        y_activity_pred[i] = 'services'

    if y_activity_pred[i] == 'Politics and affairs':
        y_activity_pred[i] = 'other'
    if y_activity_pred[i] == 'Personality and relationships':
        y_activity_pred[i] = 'other'
    if y_activity_pred[i] == 'Beliefs wishes kindness':
        y_activity_pred[i] = 'other'
  
# print list
print(y_activity_pred)

['outdoors', 'services', 'other', 'outdoors', 'other', 'other', 'transport', 'other', 'other', 'other', 'work', 'transport', 'transport', 'home', 'other', 'outdoors', 'outdoors', 'transport', 'outdoors', 'other', 'other', 'other', 'services', 'outdoors', 'other', 'outdoors', 'services', 'services', 'outdoors', 'services', 'other', 'outdoors', 'services', 'services', 'home', 'other', 'services', 'services', 'outdoors', 'services', 'transport', 'home', 'other', 'home', 'services', 'work', 'home', 'services', 'outdoors', 'services', 'work', 'home', 'services', 'home', 'other', 'home', 'outdoors', 'outdoors', 'home', 'outdoors', 'home', 'transport', 'home', 'home', 'services', 'services', 'outdoors', 'services', 'services', 'services', 'services', 'other', 'services', 'outdoors', 'home', 'transport', 'transport', 'transport', 'transport', 'outdoors', 'transport', 'transport', 'transport', 'outdoors', 'services', 'outdoors', 'home', 'services', 'work', 'work', 'other', 'services', 'other', 

In [10]:
y_activity_true = list(df['classname'].values) # true labels (annotated by human)

In [11]:
from sklearn.metrics import classification_report, confusion_matrix

In [12]:
# Classification report for the general categories (1 row left unannotated, will be fixed)
print(classification_report(y_activity_true, y_activity_pred))

              precision    recall  f1-score   support

        home       0.00      0.00      0.00         6
       other       0.30      0.38      0.33        16
    outdoors       0.20      0.40      0.27        10
    services       0.65      0.40      0.50        42
   transport       0.54      0.58      0.56        12
        work       0.57      0.29      0.38        14

    accuracy                           0.38       100
   macro avg       0.38      0.34      0.34       100
weighted avg       0.49      0.38      0.41       100



In [31]:
# Transforming list of predictions back to dataframe
df_result = pd.DataFrame(activity_result)

In [32]:
# Checking predictions
df_result.head()

,sequence,labels,scores
0,(Senior) Digital Product Designer in #Leipzig ...,"[Work job training, Discussion, Gym, Culture a...","[0.1974041610956192, 0.10330083966255188, 0.08..."
1,Jugging down my 2nd Liter of coffee for today....,"[Café restaurants drinks, Discussion, Culture ...","[0.11716744303703308, 0.09599005430936813, 0.0..."
2,Happy Weekend ?? #Metz #Lorraine https://t.co/...,"[City sightseeing, Culture and entertainment, ...","[0.18392527103424072, 0.1045927107334137, 0.06..."
3,"Good morning, Vancouver! :) #Sunrise 07:13, no...","[Discussion, Departures delays arrivals, City ...","[0.08371323347091675, 0.07710594683885574, 0.0..."
4,"Not good, not bad...disappointing\r\n#MartinaP...","[Discussion, Reading and listening, Departures...","[0.11798880994319916, 0.08938255906105042, 0.0..."


In [33]:
# Saving prediction results to csv
df_result.to_csv('result.csv')